In [25]:
import os

In [26]:
%pwd


'd:\\Text-Summerization\\research'

In [27]:
os.chdir("../")

In [28]:
%pwd

'd:\\Text-Summerization'

In [29]:
from dataclasses import dataclass
from pathlib import  Path

@dataclass(frozen=True)
class DataInjestionConfig:
    root_dir :Path
    source_URL :str
    local_data_file :Path
    unzip_dir:Path

In [30]:
from Textsummerizer.constant import *
from Textsummerizer.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataInjestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])
        data_injestion_config = DataInjestionConfig(
             root_dir=config.root_dir,
             source_URL= config.source_URL,
             local_data_file= config.local_data_file,
             unzip_dir= config.unzip_dir
                            
            )
        return data_injestion_config
    
    






In [31]:
import os
import urllib.request as request
import zipfile
from Textsummerizer.logging import logger
from Textsummerizer.utils.common import get_size

In [32]:
class DataInjestion:
    def __init__(self, config: DataInjestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers =request.urlretrieve(
                url =self.config.source_URL,
                filename =self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """zip_file_path:str
        Exctarcts the zip file into the data directory
        Function returns none
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [33]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataInjestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2024-05-04 17:12:18,644: INFO: common: yaml file : config\config.yaml loaded successfully]
[2024-05-04 17:12:18,650: INFO: common: yaml file : params.yaml loaded successfully]
[2024-05-04 17:12:18,652: INFO: common: created directory at : artifacts]
[2024-05-04 17:12:18,653: INFO: common: created directory at : artifacts/data_ingestion]
[2024-05-04 17:12:30,532: INFO: 2641548507: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C7BA:185A6F:245A0C:2EAA37:66361F1A
Accept-Ranges: bytes
Date: Sat, 04 May 2024 11:42:19 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4737-BOM
X-Cache